<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.

  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.

  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.


-Limpieza de Datos:

- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

In [2]:
import pandas as pd 
import psycopg2

1 Carga de los datos

In [3]:
df_dda_electrica = pd.read_csv("../../datos_extraccion/df_dda.csv", index_col=0)
df_gen_electrica = pd.read_csv("../../datos_extraccion/df_generacion.csv",index_col=0)
df_datos_demograficos = pd.read_csv("../../datos_extraccion/df_demo_ine.csv",index_col=0)
df_datos_eco = pd.read_csv("../../datos_extraccion/df_datos_economicos_ine.csv",index_col=0)

# Tratamiento de datos


# En el dataframe de demanda eléctrica vamos a eliminar las filas desconocidas y vamos asignarle un timestamp a la fecha de la demanda. Eliminarmos el formato float de energia_dda

In [4]:
df_dda_electrica["fecha_dda"] = pd.to_datetime(df_dda_electrica["fecha_dda"]).dt.tz_localize(None)
df_dda_electrica["energía_dda"] = df_dda_electrica["energía_dda"].replace(".","").astype(int)
filtro_desconocido = df_dda_electrica[(df_dda_electrica["ccaa"] == "Desconocido")]

In [5]:
df_dda_electrica.reset_index(inplace=True)

In [6]:
df_dda_electrica.head()

,index,ccaa,fecha_dda,energía_dda
0,0,Pais_Vasco,2019-01-01,16284370
1,1,Pais_Vasco,2020-01-01,14937556
2,2,Asturias,2019-01-01,9393095
3,3,Asturias,2020-01-01,8724907
4,4,Desconocido,2019-01-01,206048


In [7]:
df_dda_electrica.to_csv("../../datos_extraccion/dda_electrica_preparado.csv")

# Hacemos generación eléctrica

In [8]:
df_gen_electrica["fecha_generacion"] = pd.to_datetime(df_gen_electrica["fecha_generacion"]).dt.tz_localize(None)

filtro_desconocido = df_gen_electrica[(df_gen_electrica["ccaa"] == "Desconocido")]
df_gen_electrica = df_gen_electrica.drop(index = filtro_desconocido.index)
df_gen_electrica = df_gen_electrica.drop(columns = ["index"])

In [9]:
df_gen_electrica["ccaa"].unique()

array(['Andalucia', 'Aragon', 'Cantabria', 'C_LM', 'C_leon', 'Cataluña',
       'Pais_Vasco', 'Asturias', 'Madrid', 'Navarra', 'Valencia',
       'Extremadura', 'Galicia', 'La_rioja', 'Murcia'], dtype=object)

In [10]:
df_gen_electrica.to_csv("../../datos_extraccion/gen_electrica_preparado.csv")

# Hacemos datos demográficos

In [11]:
df_datos_demograficos.head()

,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total,codigo,provincia
0,0-15 años,Españoles,Ambos sexos,2022,51.924,2,Albacete
1,0-15 años,Españoles,Ambos sexos,2021,52.851,2,Albacete
2,0-15 años,Españoles,Ambos sexos,2020,53.932,2,Albacete
3,0-15 años,Españoles,Ambos sexos,2019,55.028,2,Albacete
4,0-15 años,Españoles,Hombres,2022,26.845,2,Albacete


Como los datos de la API no tenían datos por provincia, generaremos una nueva columna desde la que unir con las CCAA

In [12]:
dic_provincias_comunidades = {
    "Coruña,":"Galicia",
    "Araba/Álava": "Pais_Vasco",
    "Albacete": "C_LM",
    "Alicante/Alacant": "Valencia",
    "Almería": "Andalucia",
    "Asturias": "Asturias",
    "Ávila": "C_leon",
    "Badajoz": "Extremadura",
    "Baleares": "Islas Baleares",
    "Barcelona": "Cataluña",
    "Burgos": "C_leon",
    "Cáceres": "Extremadura",
    "Cádiz": "Andalucia",
    "Cantabria": "Cantabria",
    "Castellón/Castelló": "Valencia",
    "Ciudad": "C_LM",
    "Córdoba": "Andalucia",
    "Cuenca": "C_LM",
    "Girona": "Cataluña",
    "Granada": "Andalucia",
    "Guadalajara": "C_LM",
    "Gipuzkoa": "Pais_Vasco",
    "Huelva": "Andalucia",
    "Aragón":"Aragon",
    "Huesca": "Aragon",
    "Jaén": "Andalucia",
    "Rioja,": "La_rioja",
    "Las Palmas": "Canarias",
    "León": "C_leon",
    "Lleida": "Cataluña",
    "Lugo": "Galicia",
    "Madrid": "Madrid",
    "Málaga": "Andalucia",
    "Murcia": "Murcia",
    "Navarra": "Navarra",
    "Ourense": "Galicia",
    "Palencia": "C_leon",
    "Pontevedra": "Galicia",
    "Salamanca": "C_leon",
    "Santa": "Canarias",
    "Segovia": "C_leon",
    "Sevilla": "Andalucia",
    "Soria": "C_leon",
    "Tarragona": "Cataluña",
    "Teruel": "Aragon",
    "Toledo": "C_LM",
    "Valencia": "Valencia",
    "Valladolid": "C_leon",
    "Bizkaia": "Pais_Vasco",
    "Zamora": "C_leon",
    "Zaragoza": "Aragon",
    "Ceuta": "Ceuta",
    "Melilla": "Melilla",
    "Valencia/València": "Valencia"
}


In [13]:
df_datos_demograficos["Edad (3 grupos de edad)"].unique()
df_datos_demograficos.rename({"Edad (3 grupos de edad)":"edad"}, inplace = True)
df_datos_demograficos["ccaa"] = df_datos_demograficos["provincia"].replace(dic_provincias_comunidades)

In [14]:
df_datos_demograficos["ccaa"].unique()

array(['C_LM', 'Valencia', 'Andalucia', 'Pais_Vasco', 'Asturias',
       'C_leon', 'Extremadura', 'Balears,', 'Cataluña', 'Cantabria',
       'Galicia', 'Aragon', 'Madrid', 'Murcia', 'Navarra', 'Palmas,',
       'La_rioja', 'Canarias', 'Ceuta', 'Melilla'], dtype=object)

In [15]:
diferencia = list(df_datos_demograficos["ccaa"][~df_datos_demograficos["ccaa"].isin(df_gen_electrica["ccaa"])].unique())

filtro_diferencia = df_datos_demograficos[df_datos_demograficos["ccaa"].isin(diferencia)]

df_datos_demograficos = df_datos_demograficos.drop(index= filtro_diferencia.index)

In [16]:
df_datos_demograficos.head()

,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total,codigo,provincia,ccaa
0,0-15 años,Españoles,Ambos sexos,2022,51.924,2,Albacete,C_LM
1,0-15 años,Españoles,Ambos sexos,2021,52.851,2,Albacete,C_LM
2,0-15 años,Españoles,Ambos sexos,2020,53.932,2,Albacete,C_LM
3,0-15 años,Españoles,Ambos sexos,2019,55.028,2,Albacete,C_LM
4,0-15 años,Españoles,Hombres,2022,26.845,2,Albacete,C_LM


In [17]:
df_datos_demograficos.to_csv("../../datos_extraccion/datos_demograficos_preparado.csv")

# Le aplicamos la misma transformación a los datos economicos

In [305]:
df_datos_eco.head()

,periodo,Total,codigo,provincia
0,2021,8.663.814,2,Albacete
1,2020,7.855.556,2,Albacete
2,2019,8.475.070,2,Albacete
3,2021,37.279.735,3,Alicante/Alacant
4,2020,33.979.911,3,Alicante/Alacant


In [ ]:
diferencia = list(df_datos_eco["ccaa"][~df_datos_eco["ccaa"].isin(df_gen_electrica["ccaa"])].unique())

filtro_diferencia = df_datos_eco[df_datos_eco["ccaa"].isin(diferencia)]

df_datos_eco = df_datos_eco.drop(index= filtro_diferencia.index)

df_datos_eco["Total"] = df_datos_eco["Total"].astype(str)
df_datos_eco["Total"] = df_datos_eco["Total"].str.replace(".","").astype(int)

df_datos_eco.rename(columns = {"periodo":"año"}, inplace = True)

df_datos_eco["ccaa"] = df_datos_eco["provincia"].replace(dic_provincias_comunidades)

In [320]:
df_datos_eco.head()

,año,Total,codigo,provincia,ccaa
0,2021,8663814,2,Albacete,C_LM
1,2020,7855556,2,Albacete,C_LM
2,2019,8475070,2,Albacete,C_LM
3,2021,37279735,3,Alicante/Alacant,Valencia
4,2020,33979911,3,Alicante/Alacant,Valencia


In [325]:
df_datos_eco.to_csv("../../datos_extraccion/datos_economicos_preparado.csv")